In [55]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

sys.path.append('..')
from audio_toolbox.metrics import splitTabularPredicting

audio_root_folder = '../archive/Data'
labels_csv = os.path.join(audio_root_folder, 'data.csv')
unslicedData = pd.read_csv(labels_csv, header=0)
slicedCSV = os.path.join(audio_root_folder, 'features.csv')
slicedDataSet = pd.read_csv(slicedCSV, header=0)
label = unslicedData['label']
unslicedData = unslicedData.sample(frac=1).reset_index(drop=True)
pca = PCA(n_components=0.90)

In [56]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()

In [57]:
#generate the testing data folds
K = 5
testDataFolds = []
length = len(unslicedData)
slice_size = length // K
start_index = 0
for i in range(K):
    end_index = min(start_index + slice_size, length)
    fold = unslicedData.iloc[start_index: end_index]
    start_index = end_index
    testDataFolds.append(fold)

In [58]:
#generate the training data folds
trainData = []
for i in range(K):
    index = testDataFolds[i]['index']
    trainData.append(slicedDataSet[~slicedDataSet['index'].isin(index)])

In [59]:
#model training
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#model = RandomForestClassifier(max_depth = 15, max_features = 8)
#model = LogisticRegression(max_iter=1000, C=1e-3)
model = SVC(kernel='rbf', C=1)

predictions = []
testLabels = []
for i in range(K):
    X_train = trainData[i].copy()
    label = X_train['label']
    X_train.drop(['label','index'], axis=1, inplace=True)
    X_train_scaled = pd.DataFrame(scalar.fit_transform(X_train), columns=X_train.columns)
    X_train_pca = pca.fit_transform(X_train_scaled)
    #npArray = scalar.fit_transform(npArray)
    #X_train = pd.DataFrame(npArray, columns=X_train.columns)
    #trainingData, _, trainingLabel, _ = train_test_split(X_train_pca, label, train_size=0.5)
    #model.fit(trainingData, trainingLabel)
    model.fit(X_train_pca, label)
    #model.fit(X_train_scaled, label)
    #trainAccuracy = accuracy_score(model.predict(X_train_scaled), label)
    #trainAccuracy = accuracy_score(model.predict(X_train), label)
    trainAccuracy = accuracy_score(model.predict(X_train_pca), label)
    print(f"training accuracy: {trainAccuracy}")
    testData = testDataFolds[i].copy()
    testIndex = testData['index']
    testLabel = testData['label']
    testLabels.append(testLabel)
    #dataArray = testData.values
    #dataArray = scalar.transform(dataArray)
    #testData = pd.DataFrame(dataArray, columns=testData.columns)
    prediction= splitTabularPredicting(model, testIndex, slicedDataSet, scalar, pca)
    predictions.append(prediction)

training accuracy: 0.9424496434380083
training accuracy: 0.9490931832395247
training accuracy: 0.9430680680680681
training accuracy: 0.9470720720720721
training accuracy: 0.9465715715715716


In [60]:
#average accuracy
accuracies = []
for i in range(K):
    prediction = predictions[i]
    accuracy = accuracy_score(testLabels[i], prediction)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy: {mean_accuracy}")

Mean Accuracy: 0.7658291457286432
